# Multimodal Calssification on Rakuten France Dataset
# Multi Modal Addition Fusion

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# !ls

In [6]:
# pwd

In [7]:
# mkdir Rakuten

In [8]:
# cd './Rakuten'

In [9]:
mkdir models data 

mkdir: cannot create directory ‘models’: File exists
mkdir: cannot create directory ‘data’: File exists


In [10]:
pwd

'/content/Rakuten'

In [11]:
# cp '/content/drive/My Drive/Rakuten/models/CamemBERT_best_model_split_title.pt' models

In [12]:
# cp '/content/drive/My Drive/Rakuten/models/CamemBERT_best_model_split_description.pt' models


In [13]:
# cp '/content/drive/My Drive/Rakuten/models/FlauBERT_best_model_split_description.pt' models

In [14]:
# cp '/content/drive/My Drive/Rakuten/models/FlauBERT_best_model_split_title.pt' models


In [15]:
# cp '/content/drive/My Drive/Rakuten/models/Final_RESNet_model.pt' models

In [16]:
# pwd

In [ ]:
!cp  '/content/drive/My Drive/Rakuten/image.zip' './'

In [ ]:
ls -la

In [ ]:
!unzip  ./image.zip

In [ ]:
pwd

In [ ]:
 !cp '/content/drive/My Drive/Rakuten/data/NewTest.csv' data 
 !cp '/content/drive/My Drive/Rakuten/data/NewTraining.csv' data 
 !cp '/content/drive/My Drive/Rakuten/data/catalog_english_taxonomy.tsv' data 
 !cp '/content/drive/My Drive/Rakuten/data/Y_train.tsv' data 
 !cp '/content/drive/My Drive/Rakuten/data/X_train.tsv' data

In [ ]:
pwd

# 1. Setup

# 1.1 Using Colab GPU for Training

Since we’ll be training a large neural network it’s best to take advantage of the free GPUs and TPUs that Google offers (in this case we’ll attach a GPU), otherwise training will take a very long time.

A GPU can be added by going to the menu and selecting:

Edit 🡒 Notebook Settings 🡒 Hardware accelerator 🡒 (GPU)

In [ ]:
import os, time, datetime
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import random
import logging
tqdm.pandas()
import seaborn as sns
from sklearn.model_selection import train_test_split
from google.colab import drive

#NN Packages
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, random_split,DataLoader, RandomSampler, SequentialSampler

logger = logging.getLogger(__name__)

In [ ]:
torch.manual_seed(123)
torch.cuda.manual_seed(123)
torch.backends.cudnn.enabled=False
torch.backends.cudnn.deterministic=True

In [ ]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

# 1.2. Installing the Hugging Face Library - Image Pretrained Models

Install the transformers package from **Hugging Face** which will give us a pytorch interface for working with BERT. This library contains interfaces for other pretrained language models.

We’ve selected the pytorch interface because it strikes a nice balance between the high-level APIs (which are easy to use but don’t provide insight into how things work) and tensorflow code (which contains lots of details but often sidetracks us into lessons about tensorflow, when the purpose here is BERT!).

At the moment, the Hugging Face library seems to be the most widely accepted and powerful pytorch interface for working with BERT. In addition to supporting a variety of different pre-trained transformer models, the library also includes pre-built modifications of these models suited to your specific task.
E.g "BertForSequenceClassification" that we will be using.

The goal  of the **pretrainedmodels** is to:

-  help to reproduce transfer learning setups

-  access pretrained ConvNets with a unique interface/API inspired by torchvision.

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install pretrainedmodels

# 2. Dataset Loading and Preprocessing

# 2.1 Dataset Loading

In [ ]:
text_data_path = '/content/Rakuten/data'
image_data_path = '/content/Rakuten/image' 

In [ ]:
class SigirPreprocess():

  def __init__(self, text_data_path):
    
        self.text_data_path = text_data_path
        self.train = None # Merged X_train and Y_train
        self.dict_code_to_id = {}
        self.dict_id_to_code = {}
        self.list_tags = {} #unique type code
        self.sentences = []
        self.labels = []
        self.text_col = None
        self.X_test = None

  def prepare_data(self):
        
        # #loading the Merged, preprocessed text data and test data
        # train = pd.read_csv(self.text_data_path+"/NewTraining.csv")
        # # new_train =  train[train['Description'] != " "]
        # # new_train = new_train[new_train['Description'].notna()]
        # self.train = train

        catalog_eng= pd.read_csv(text_data_path+"/catalog_english_taxonomy.tsv",sep="\t")
        X_train= pd.read_csv(text_data_path+"/X_train.tsv",sep="\t")
        Y_train= pd.read_csv(text_data_path+"/Y_train.tsv",sep="\t")
        
        self.list_tags = list(Y_train['Prdtypecode'].unique())
        for i,tag in enumerate(self.list_tags):
            self.dict_code_to_id[tag] = i 
            self.dict_id_to_code[i]=tag
        print(self.dict_code_to_id)
            
        Y_train['labels']=Y_train['Prdtypecode'].map(self.dict_code_to_id)
        train=pd.merge(left=X_train,right=Y_train,
               how='left',left_on=['Integer_id','Image_id','Product_id'],
               right_on=['Integer_id','Image_id','Product_id'])
        prod_map=pd.Series(catalog_eng['Top level category'].values,
                           index=catalog_eng['Prdtypecode']).to_dict()

        train['product'] = train['Prdtypecode'].map(prod_map)
        train['title_len']=train['Title'].progress_apply(lambda x : len(x.split()) if pd.notna(x) else 0)
        train['desc_len']=train['Description'].progress_apply(lambda x : len(x.split()) if pd.notna(x) else 0)
        train['title_desc_len']=train['title_len'] + train['desc_len']
        train.loc[train['Description'].isnull(), 'Description'] = " "
        train['title_desc'] = train['Title'] + " " + train['Description']
        
        self.train = train

        
  def get_sentences(self, text_col, remove_null_rows=True):

       #get values of a specific column
        self.text_col = text_col        

        new_train = self.train.copy()  
        self.sentences = new_train[text_col].values
        self.labels = new_train['labels'].values


  def prepare_test(self, text_col):
    
        X_test = pd.read_csv(self.text_data_path + "/NewTest.csv")
        self.X_test = X_test
        X_test['title_desc'] = X_test['Title'] + " " + X_test['Description']
        self.test_sentences = X_test[text_col].values
        return self.test_sentences
        

# 2.2 Drop Records With No Description

In [ ]:
#Load train and test data (test for specific column)

text_col = 'title_desc'

max_len = 256

num_classes = 27

Preprocess = SigirPreprocess(text_data_path)

Preprocess.prepare_data()
train = Preprocess.train
# print("Trian:  ", len(Preprocess.train))


Preprocess.get_sentences(text_col)
# print("Labels: ", len(Preprocess.labels))

# X_test = Preprocess.prepare_test(text_col)
# print("Test:   ", len(Preprocess.X_test))

In [ ]:
from bs4 import BeautifulSoup
Preprocess.train['Description'] = [BeautifulSoup(text).get_text() for text in  Preprocess.train['Description'] ]
Preprocess.train['Title'] = [BeautifulSoup(text).get_text() for text in  Preprocess.train['Title'] ]
Preprocess.train['title_desc'] = [BeautifulSoup(text).get_text() for text in  Preprocess.train['title_desc'] ]


In [ ]:
from collections import Counter
import matplotlib.pyplot as plt


counter = Counter(Preprocess.labels)
for k,v in counter.items():
	per = v / len(Preprocess.labels) * 100
	print('Class=%d, n=%d (%.3f%%)' % (k, v, per))

print("")
# plot the distribution
plt.bar(counter.keys(), counter.values())
plt.show()

In [ ]:
print(Preprocess.train['Title'].isnull().sum())

print(Preprocess.train['Description'].isnull().sum())

print(Preprocess.train['Image_id'].isnull().sum())

print(Preprocess.train['Product_id'].isnull().sum())

print(Preprocess.train['Prdtypecode'].isnull().sum())

print(Preprocess.train['labels'].isnull().sum())

print(Preprocess.train['product'].isnull().sum()) #top level category

print(Preprocess.train['title_desc'].isnull().sum())




In [ ]:
Preprocess.train.isnull().values.any()

In [ ]:
Preprocess.train.isnull().sum().sum()

In [ ]:
Preprocess.train

# View Tokenizer Input 

In [ ]:
Preprocess.get_sentences(text_col,True)
sentences = Preprocess.sentences

labels = Preprocess.labels
print(sentences)


In [ ]:
print (type(sentences))
print()
# print("Total number of sentences:{}, labels:{}".format(len(sentences), len(labels)))

# View Test Sentences

In [ ]:
# batch_size = 32  

# test_sentences = Preprocess.test_sentences

# X_test_phase1  = Preprocess.X_test

In [ ]:
# print(len(test_sentences))

**Helper Function**

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

# 3. Tokenization & Input Formatting
 Transform our dataset into the format that BERT can be trained on.

# 3.1. BERT Tokenizer
To feed our text to BERT, it must be split into tokens, and then these tokens must be mapped to their index in the tokenizer vocabulary.

The tokenization must be performed by the tokenizer included within BERT

In [ ]:
from transformers import XLMForSequenceClassification
from transformers import FlaubertModel, FlaubertTokenizer,FlaubertForSequenceClassification,AdamW, FlaubertConfig 

from torch.nn import Dropout,Conv1d, Linear
from transformers.modeling_utils import SequenceSummary

#from transformers.modeling_roberta import RobertaClassificationHead


# 3.2. Required Formatting
We are required to give it a number of pieces of information

We need to:

Add special tokens to the start and end of each sentence.
Pad & truncate all sentences to a single constant length.
Explicitly differentiate real tokens from padding tokens with the “attention mask”.

# 3.3. Tokenize Dataset
We will use "encode_plus":

returns a dictionary containing the encoded sequence or sequence pair and additional information: the mask for sequence classification and the overflowing elements if a max_length is specified.

In [ ]:
def prep_input(sentences,labels, max_len,tokenizer):
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in sentences:
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            sent,                           # Sentence to encode.
                            add_special_tokens = True,      # Add '[CLS]' and '[SEP]'
                            max_length = max_len,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )

        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])       # IDs of the the vocabularies in the Model's dictionary

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors. 
    input_ids = torch.cat(input_ids, dim=0)             # Concatenates the given sequence of seq tensors in the given dimension. 
                                                        # All tensors must  have the same shape 
    attention_masks = torch.cat(attention_masks, dim=0)

    if labels is not None:
        labels = torch.tensor(labels)
        return input_ids, attention_masks, labels
    else:
        return input_ids, attention_masks

# 3.4. Importing Tokenizers and Input Preparation

- Now it is time to import both Camembert and FlauBERT tokenizers from  pretained package and prepare the input using them. 

- Calling prep_input() for each model will result in the corresponding:
     

1.   **input ids**
2.   **attention maks**
3.   **labels**

 

In [ ]:
from transformers import CamembertConfig, CamembertTokenizer, CamembertModel, CamembertForSequenceClassification, AdamW
from transformers import FlaubertModel, FlaubertTokenizer,FlaubertForSequenceClassification,AdamW, FlaubertConfig 

print('Using Camembert')
tokenizer_cam = CamembertTokenizer.from_pretrained('camembert-base', do_lowercase=False)
print('Using Flaubert')
tokenizer_flau = FlaubertTokenizer.from_pretrained('flaubert/flaubert_base_cased', do_lowercase=False)

In [ ]:
input_ids_cam, attention_masks_cam, labels_cam = prep_input (sentences, labels, max_len, tokenizer_cam)


In [ ]:
input_ids_flau, attention_masks_flau, labels_flau  = prep_input(sentences,labels, max_len,tokenizer_flau)

# 3.5. Training & Validation Split
Divide up our training randomly select **10%** as a validation set off of the training set.

While splitting, we used the following parameters:


1.   **stratify**: 
in this context, stratification means that the train_test_split method returns training and test subsets that have the same proportions of class labels as the input dataset.
2.   **random_state**: 
simply sets a seed to the random generator, so that your train-test splits are always deterministic. If you don't set a seed, it is different each time.

In [ ]:
# val_size = 0.15
#basic ---------------

# tr_inputs_cam, val_inputs_cam, _,_ = train_test_split (input_ids_cam, labels_cam, stratify = labels_cam,    
#                                                             random_state=2020, test_size = val_size)

# tr_masks_cam, val_masks_cam, _,_ =   train_test_split (attention_masks_cam, labels, stratify = labels,        # labels: Preprocess.labels
#                                                             random_state=2020, test_size = val_size)


# tr_inputs_flau, val_inputs_flau, _,_ = train_test_split (input_ids_flau, labels_flau, stratify=labels,
#                                                             random_state=2020, test_size = val_size)

# tr_masks_flau, val_masks_flau, _,_   = train_test_split (attention_masks_flau, labels,stratify=labels_flau,  # labels: Preprocess.labels
#                                                             random_state=2020, test_size = val_size)

In [ ]:
# tr_inputs, test_inputs_cam, tr_labels, test_labels_cam = train_test_split(input_ids_cam, labels_cam, stratify=labels_cam, random_state=2020,
#                                                                 test_size = 0.2)

# tr_inputs_cam, val_inputs_cam, train_labels, val_labels = train_test_split(tr_inputs, tr_labels, stratify=tr_labels, random_state=2020,
#                                                                 test_size = 0.15)

# tr_masks, test_masks_cam, tr_masks_labels, _ =   train_test_split(attention_masks_cam, labels, stratify=labels, random_state=2020,
#                                                                  test_size=0.2)

# tr_masks_cam, val_masks_cam, u,v =   train_test_split(tr_masks, tr_masks_labels, stratify=tr_masks_labels, random_state=2020,
#                                                                 test_size=0.15 )




# tr_inputs, test_inputs_flau, tr_labels, test_labels_flau = train_test_split(input_ids_flau, labels_flau, stratify=labels_cam, random_state=2020,
#                                                                 test_size = 0.2)

# tr_inputs_flau, val_inputs_flau, train_labels, val_labels = train_test_split(tr_inputs, tr_labels, stratify=tr_labels, random_state=2020,
#                                                                 test_size = 0.15)

# tr_masks, test_masks_flau, tr_masks_labels, _ =   train_test_split(attention_masks_flau, labels, stratify=labels, random_state=2020,
#                                                                  test_size=0.2)

# tr_masks_flau, val_masks_flau, u,v =   train_test_split(tr_masks, tr_masks_labels, stratify=tr_masks_labels, random_state=2020,
#                                                                 test_size=0.15 )

In [ ]:
tr_inputs, val_inputs_cam, tr_labels, val_labels_cam = train_test_split(input_ids_cam, labels_cam, stratify=labels_cam, random_state=2020,
                                                                test_size = 0.2)

tr_inputs_cam, test_inputs_cam, train_labels, test_labels = train_test_split(tr_inputs, tr_labels, stratify=tr_labels, random_state=2020,
                                                                test_size = 0.15)

tr_masks, val_masks_cam, tr_masks_labels, _ =   train_test_split(attention_masks_cam, labels, stratify=labels, random_state=2020,
                                                                 test_size=0.2)

tr_masks_cam, test_masks_cam, u,v =   train_test_split(tr_masks, tr_masks_labels, stratify=tr_masks_labels, random_state=2020,
                                                                test_size=0.15 )




tr_inputs, val_inputs_flau, tr_labels, val_labels_flau = train_test_split(input_ids_flau, labels_flau, stratify=labels_cam, random_state=2020,
                                                                test_size = 0.2)

tr_inputs_flau, test_inputs_flau, train_labels, test_labels = train_test_split(tr_inputs, tr_labels, stratify=tr_labels, random_state=2020,
                                                                test_size = 0.15)

tr_masks, val_masks_flau, tr_masks_labels, _ =   train_test_split(attention_masks_flau, labels, stratify=labels, random_state=2020,
                                                                 test_size=0.2)

tr_masks_flau, test_masks_flau, u,v =   train_test_split(tr_masks, tr_masks_labels, stratify=tr_masks_labels, random_state=2020,
                                                                test_size=0.15 )

In [ ]:
torch.save(tr_inputs_cam, "tr_inputs_cam.pt")
torch.save(val_inputs_cam, "val_inputs_cam.pt")
torch.save(tr_masks_cam, "tr_masks_cam.pt")
torch.save(val_masks_cam, "val_masks_cam.pt")
torch.save(test_inputs_cam, "test_inputs_cam.pt")
torch.save(test_masks_cam, "test_masks_cam.pt")


torch.save(tr_inputs_flau, "tr_inputs_flau.pt")
torch.save(val_inputs_flau, "val_inputs_flau.pt")
torch.save(tr_masks_flau, "tr_masks_flau.pt")
torch.save(val_masks_flau, "val_masks_flau.pt")
torch.save(test_inputs_flau, "test_inputs_flau.pt")
torch.save(test_masks_flau, "test_masks_flau.pt")



In [ ]:
text_input='./'

tr_inputs_cam = torch.load(text_input + "tr_inputs_cam.pt")
val_inputs_cam = torch.load(text_input +"val_inputs_cam.pt")
tr_masks_cam = torch.load(text_input + "tr_masks_cam.pt")
val_masks_cam = torch.load(text_input + "val_masks_cam.pt")
input_ids_test_cam = torch.load(text_input + "test_inputs_cam.pt") 
attention_masks_test_cam = torch.load(text_input + "test_masks_cam.pt") 

tr_inputs_flau = torch.load(text_input + "tr_inputs_flau.pt")
val_inputs_flau = torch.load(text_input + "val_inputs_flau.pt")
tr_masks_flau = torch.load(text_input + "tr_masks_flau.pt")
val_masks_flau = torch.load(text_input + "val_masks_flau.pt")
input_ids_test_flau = torch.load(text_input + "test_inputs_flau.pt")
attention_masks_test_flau = torch.load(text_input + "test_masks_flau.pt")

# 4. Defining Models to be Fused

Now, as our  data has been preprocessed, cleaned and text data was tokenzied , it is ready to be fed to the models. 
- As a first step,  first we need to define and configure the models. 


# 4.1. RESNet Model for Image Processing. 

In PyTorch, you always need to define a forward method for your neural network model. But you never have to call it explicitly.
Here we are defining our image processing class is subclass of nn.Module and is inheriting all methods. In the super class, nn.Module, there is a __call__ method which obtains the forward function from the subclass and calls it.

# 4.1.1.  Image Processing Model - RESNet50




In [ ]:
from torch.nn import functional as F
import torch.nn as nn
import pretrainedmodels
class SEResnext50_32x4d(nn.Module):
    def __init__(self, pretrained='imagenet'):
        super(SEResnext50_32x4d, self).__init__()
        
        self.base_model = pretrainedmodels.__dict__["se_resnext50_32x4d"](pretrained=None)
        if pretrained is not None:
            self.base_model.load_state_dict(
                self.base_model.load_state_dict(torch.load(resnet_model_path))
                )
            
        self.l0 = nn.Linear(2048, 27)  # Applies a linear transformation to the incoming data
        # batch_size = 2048
    
    def forward(self, image):
        batch_size, _, _, _ = image.shape

        # During the training you will get batches of images, 
        # so your shape in the forward method will get an additional batch dimension at dim0: 
        # [batch_size, channels, height, width].
        
        x = self.base_model.features(image) 

        #extracting feature vector from network after feature leaning 
        #This is the flatten vector 

        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1) 
        #adaptive_avg_pool2d : Kernel size = (input_size+target_size-1) // target_size rounded up
        #Then the positions of where to apply the kernel are computed as rounded equidistant points between 0 and input_size - kernel_size
        
        out = self.l0(x)

        return out

In [ ]:
class Identity(nn.Module):
  
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

# 4.1.2. Instaniating the Image Processing Network 
 Now we create an instance from the SEResnext50_32x4d class that we defined and load the weights from a pretrained model, since the training is done previously. 

In [ ]:
#data_path = '/content/drive/My Drive/Rakuten/'

img_model = SEResnext50_32x4d(pretrained=None)
# img_model.load_state_dict(torch.load(os.path.join(data_path, 'models/RESNET_best_model.pt')))

# img_model.cuda()

# 4.1.3. Prinitng Model's Params

In [ ]:
img_model.l0 = Identity()

In [ ]:
for param in img_model.parameters():
     print(type(param), param.size())

# 4.1.4. Model's Params Require No Grads

These are just regular tensors, with one very special addition: we tell PyTorch that they require a gradient. This causes PyTorch to record all of the operations done on the tensor, so that it can calculate the gradient during back-propagation automatically!
As our model is already trained and weights are assigned, then there is no need to calculate the gradients so no need to send them to the optimizer.

In [ ]:
# params are iterators which contain model's parameters. Usually passed to the optimizer
for params in img_model.parameters():
      params.requires_grad = False

In [ ]:
img_model.out_proj = Identity()

# Image Data Preparation

In [ ]:
# # Data path
# text_data_path = os.path.join('/content/drive/My Drive/Rakuten')
# image_data_path = os.path.join('')


In [ ]:
def get_img_path(img_id, prd_id, path):
    
    pattern = 'image'+'_'+str(img_id)+'_'+'product'+'_'+str(prd_id)+'.jpg'
    return path + pattern

In [ ]:
# print(len(Preprocess.train), len(train))

# Obtaining & Splitting Images 

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

#  train 65%
#  validation 15%
#  test 20%
# (original training) =>  train, test => 80, 20  val_size = 0.2
# (train) => train, val =>  85, 15   val_size = 0.15 


train_img = train[['Image_id','Product_id','labels','product']]

train_img['image_path'] = Preprocess.train.progress_apply(lambda x: get_img_path(x['Image_id'], x['Product_id'],
                                                      path = os.path.join(image_data_path, 'image_training/')),axis=1)




tr_df, val_df, tr_labels, val_labels = train_test_split(train_img, train_img['labels'], 
                                           random_state=2020,
                                           test_size = 0.2,
                                           stratify=train_img['labels'])


train_df, test_df, train_labels, test_labels = train_test_split(tr_df, tr_labels, 
                                           random_state=2020,
                                           test_size = 0.15,
                                           stratify=tr_labels)



print("Train: ", len(train_df))
print("Val:   ", len(val_df))
print("Test:  ", len(test_df))
print ("")

In [ ]:
print("Original Images Df:   ",  len(train_img))
print("Train Images DF:      " , len(train_df))
print("Validation Images DF: ",  len(val_df))

In [ ]:
print (train_img['image_path'][0])


# Image Data Augmentation

We're going to be making use of Pytorch's transforms for preparing the input images to be used by our model. 





*   We'll need to make sure the images in the training set and validation set are the same size, so we'll be using transforms.Resize
*   We'll also be doing a little data augmentation, trying to improve the performance of our model by forcing it to learn about images at different angles and crops, so we'll randomly crop and rotate the images.

*    we'll make tensors out of the images, as PyTorch works with tensors. 
*   Finally, we'll normalize the images, which helps the network work with values that may be have a wide range of different values.


*   We then compose all our chosen transforms.

It worth mentioning that validation transforms don't have any of the flipping or rotating, as they aren't part of our training set, so the network isn't learning about them







In [ ]:
input_size = 224 # for Resnt

# Applying Transforms to the Data

from torchvision import datasets, models, transforms

image_transforms = { 
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.Resize(size=256),
        transforms.CenterCrop(size=input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

# Text Processing Models - BertForSequenceClassification

Thankfully, the huggingface pytorch implementation includes a set of interfaces designed for a variety of NLP tasks. Though these interfaces are all built on top of a trained BERT model, each has different top layers and output types designed to accomodate their specific NLP task.

We first want to modify the pre-trained BERT model to give outputs for classification, and then we want to continue training the model on our dataset until that the entire model, end-to-end, is well-suited for our task.

**BertForSequenceClassification** is one of the current of classes provided for fine-tuning.

This is the normal BERT model with an added single linear layer on top for classification that we will use as a sentence classifier. As we feed input data, the entire pre-trained BERT model and the additional untrained classification layer is trained on our specific task.

- Not to forget that Camembet model inherits RobertaModel

# 4.2 CamemBERT Model

In [ ]:
class vec_output_CamembertForSequenceClassification(CamembertModel):
  
    config_class = CamembertConfig

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.roberta = CamembertModel(config)
        self.dense = nn.Linear(256*config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(0.1)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()


    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
    ):
        outputs = self.roberta(
            input_ids,
            attention_mask = attention_mask,
            token_type_ids = token_type_ids,
            position_ids = position_ids,
            head_mask = head_mask,
            inputs_embeds=inputs_embeds,
#           output_attentions=output_attentions,
#           output_hidden_states=output_hidden_states,
        )

        sequence_output = outputs[0] #(B,256,768)

        x = sequence_output.view(sequence_output.shape[0], 256*768)

#       x = sequence_output[:, 0, :]  # take <s> token (equiv. to [CLS])-> #(B,768) Image -> (B,2048)

        x = self.dense(x)  # 768 -> 768

        feat= torch.tanh(x) 

        logits = self.out_proj(feat) # 768 -> 27

        outputs = (logits,) + outputs[2:] #3rd element onwards

        return outputs,feat  # (loss), logits, (hidden_states), (attentions)

In [ ]:
camembert_model_path = '/content/Rakuten/models/CamemBERT_best_model_description.pt'

# FlauBERT Model

In [ ]:
num_classes = 27

class vec_output_FlaubertForSequenceClassification(FlaubertModel):
    
    config_class = FlaubertConfig
    

    def __init__(self, config):
        super().__init__(config)
        self.transformer = FlaubertModel(config)
        self.sequence_summary = SequenceSummary(config)
        self.init_weights()
        self.dropout =  torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(config.hidden_size, num_classes)


    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        langs=None,
        token_type_ids=None,
        position_ids=None,
        lengths=None,
        cache=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):
        
        
        transformer_outputs = self.transformer(
            input_ids,
            attention_mask = attention_mask,
            langs = langs,
            token_type_ids = token_type_ids,
            position_ids = position_ids,
            lengths = lengths,
            cache = cache,
            head_mask = head_mask,
            inputs_embeds = inputs_embeds,
        )

        #output = self.dropout(output)
        output = transformer_outputs[0] 
        vec = output[:,0]
        
        
        #logits
        dense = self.dropout(vec)
        
        #classifier
        logits = self.classifier(dense)
        
        outputs = (logits,) + transformer_outputs[1:]  # Keep new_mems and attention/hidden states if they are here
       
        
        return outputs,dense

# Dataset Fusion

In [ ]:
# TODO DELELTE IMAGES WITH NO DESCRIPTION
# From the preprocesssed file

In [ ]:
from torch.utils.data import Dataset, DataLoader, Subset
from PIL import Image
import matplotlib.pyplot as plt
import cv2

class FusionDataset(Dataset):
    
    def __init__(self, df, inputs_cam, masks_cam, inputs_flau, masks_flau, transform=None, mode='train'):
        self.df = df
        self.transform   = transform
        self.mode = mode

        self.inputs_cam  = inputs_cam
        self.masks_cam   = masks_cam

        self.inputs_flau  = inputs_flau
        self.masks_flau   = masks_flau
         
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        
        im_path = self.df.iloc[idx]['image_path']
        img= plt.imread(im_path)
        #img = cv2.imread(im_path)
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        img              = img.cuda()
        input_id_cam     = self.inputs_cam[idx].cuda()
        input_mask_cam   = self.masks_cam[idx].cuda()
        input_id_flau    = self.inputs_flau[idx].cuda()
        input_mask_flau  = self.masks_flau[idx].cuda()
        
        if self.mode =='test':
            return img,input_id_cam,input_mask_cam,input_id_flau,input_mask_flau
            
        else:
            labels = torch.tensor(self.df.iloc[idx]['labels']).cuda()             
            return img,input_id_cam,input_mask_cam,input_id_flau,input_mask_flau,labels

In [ ]:
a1 = torch.randn(3,10,10)

In [ ]:
reduce_dim = nn.Conv1d(in_channels = 10 , out_channels = 1 , kernel_size= 1)

In [ ]:
reduce_dim(a1).view(3,10).shape

# Test Sentences Tokenization

In [ ]:
print('Using Camembert')
tokenizer_cam = CamembertTokenizer.from_pretrained('camembert-base', do_lowercase=False)

print('Using Flaubert')
tokenizer_flau = FlaubertTokenizer.from_pretrained('flaubert/flaubert_base_cased', do_lowercase=False)

# input_ids_test_flau,attention_masks_test_flau = prep_input(test_sentences, labels=None, max_len=max_len,tokenizer = tokenizer_flau)

# input_ids_test_cam,attention_masks_test_cam = prep_input(test_sentences , labels=None, max_len=max_len,tokenizer = tokenizer_cam)

In [ ]:
# print(type(Preprocess.test_sentences))
# print(len(Preprocess.test_sentences))

In [ ]:
# # Moodels path 
# resnet_model_path = '/content/Rakuten/models/RESNET_best_model.pt'
# camembert_model_path = '/content/Rakuten/models/CamemBERT_best_model_title_description.pt' ###### TODO Change with the updated model!!!
# flaubert_model_path = '/content/Rakuten/models/FlauBERT_best_model_title_description.pt'

#my_flau_path  = '/content/Rakuten/models/FlauBERT_best_model_description.pt'


In [ ]:
# Moodels path 
resnet_model_path = '/content/Rakuten/models/Final_RESNet_model.pt'

camembert_model_path_title = '/content/Rakuten/models/CamemBERT_best_model_split_title.pt'
camembert_model_path_desc = '/content/Rakuten/models/CamemBERT_best_model_split_description.pt'

flaubert_model_path_title = '/content/Rakuten/models/FlauBERT_best_model_split_title.pt'
flaubert_model_path_desc = '/content/Rakuten/models/FlauBERT_best_model_split_description.pt'


# Fuse
 When using pretrained models, PyTorch sets the model to be unfrozen (will have its weights adjusted) by default

In [ ]:
class vector_fusion(nn.Module):    
    def __init__(self):
        super(vector_fusion, self).__init__()

        self.img_model = SEResnext50_32x4d(pretrained=None)
        self.img_model.load_state_dict(torch.load(resnet_model_path))
        self.img_model.l0=Identity()
        for params in self.img_model.parameters():
            params.requires_grad=False

# ------ CamamBERT ------

        self.cam_model_title = vec_output_CamembertForSequenceClassification.from_pretrained(
         'camembert-base', # Use the 12-layer BERT model, with an uncased vocab.
          num_labels = 27, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
          output_attentions = False, # Whether the model returns attentions weights.
          output_hidden_states = False,) # Whether the model returns all hidden-states.
        
        
        checkpoint = torch.load(camembert_model_path_title)
        self.cam_model_title.load_state_dict(checkpoint)

        for param in self.cam_model_title.parameters():
            param.requires_grad=False

        self.cam_model_title.out_proj = Identity()


        self.cam_model_desc = vec_output_CamembertForSequenceClassification.from_pretrained(
         'camembert-base', # Use the 12-layer BERT model, with an uncased vocab.
          num_labels = 27, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
          output_attentions = False, # Whether the model returns attentions weights.
          output_hidden_states = False,) # Whether the model returns all hidden-states.
        
        
        checkpoint = torch.load(camembert_model_path_desc)
        self.cam_model_desc.load_state_dict(checkpoint)

        for param in self.cam_model_desc.parameters():
            param.requires_grad=False

        self.cam_model_desc.out_proj = Identity()

 # ----  FlauBERT ----- 

        
        self.flau_model_title = vec_output_FlaubertForSequenceClassification.from_pretrained(
        'flaubert/flaubert_base_cased', 
        num_labels = 27, 
        output_attentions = False,
        output_hidden_states = False,)
        checkpoint = torch.load(flaubert_model_path_title)

        self.flau_model_title.load_state_dict(checkpoint)

        for param in self.flau_model_title.parameters():
            param.requires_grad=False

        self.flau_model_title.classifier=Identity()


      
        self.flau_model_desc = vec_output_FlaubertForSequenceClassification.from_pretrained(
        'flaubert/flaubert_base_cased', 
        num_labels = 27, 
        output_attentions = False,
        output_hidden_states = False,)
        checkpoint = torch.load(flaubert_model_path_desc)

        self.flau_model_desc.load_state_dict(checkpoint)

        for param in self.flau_model_desc.parameters():
            param.requires_grad=False

        self.flau_model_desc.classifier=Identity()


# ------------------------------------------------------------

        self.reduce_dim = nn.Conv1d(in_channels = 2048 , out_channels = 768 , kernel_size= 1)
        self.reduce_dim2 = nn.Conv1d(in_channels = 768 , out_channels = 1 , kernel_size= 1)
        self.out = nn.Linear(768*3, 27)
        
        #gamma
#         self.w1 = nn.Parameter(torch.zeros(1))
#         self.w2 = nn.Parameter(torch.zeros(1))
#         self.w3 = nn.Parameter(torch.zeros(1))
        
    def forward(self,img,input_id_cam,input_mask_cam,input_id_flau,input_mask_flau):
        
        cam_emb_title,vec1_title = self.cam_model_title(input_id_cam, 
                     token_type_ids=None, 
                     attention_mask=input_mask_cam)
        
        cam_emb_desc,vec1_desc = self.cam_model_desc(input_id_cam, 
                     token_type_ids=None, 
                     attention_mask=input_mask_cam)
        

#---------------------------------
        
        flau_emb_title,vec2 =self.flau_model_title(input_id_flau, 
                     token_type_ids=None, 
                     attention_mask=input_mask_flau)
        
        flau_emb_desc,vec2_desc =self.flau_model_desc(input_id_flau, 
                     token_type_ids=None, 
                     attention_mask=input_mask_flau)
        
# ---------------------------------
        
        #Projecting the image embedding to lower dimension
        img_emb = self.img_model(img)
        
        img_emb = img_emb.view(img_emb.shape[0],img_emb.shape[1],1)
        img_emb = self.reduce_dim(img_emb)
        img_emb = img_emb.view(img_emb.shape[0],img_emb.shape[1]) ###### bs * 768 
# --------------------------------

        #summing up the vectors
        cam_emb  = cam_emb_title + cam_emb_desc
        flau_emb = flau_emb_title + flau_emb_desc
        
        #Bilinear
        #text_emb = text_emb.view(text_emb.shape[0],1,text_emb.shape[1])  ##### bs * 1 * 768
        
        #Bilinear Pooling
        #pool_emb = torch.bmm(img_emb,text_emb) ### bs * 768 * 768
        #pool_emb = self.reduce_dim2(pool_emb).view(text_emb.shape[0],768)  #### bs * 1 * 768
        fuse= torch.cat([img_emb,cam_emb[0],flau_emb[0]],axis=1)
        
        
        logits = self.out(fuse)
        return logits

In [ ]:
 img_model = SEResnext50_32x4d(pretrained=None)
 img_model.load_state_dict(torch.load(resnet_model_path))

In [ ]:
cam_title= vec_output_CamembertForSequenceClassification.from_pretrained(
         'camembert-base', # Use the 12-layer BERT model, with an uncased vocab.
          num_labels = 27, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
          output_attentions = False, # Whether the model returns attentions weights.
          output_hidden_states = False,) # Whether the model returns all hidden-states.
        
        
checkpoint = torch.load(camembert_model_path_title)
cam_title.load_state_dict(checkpoint)

In [ ]:
# cam_desc= vec_output_CamembertForSequenceClassification.from_pretrained(
#          'camembert-base', # Use the 12-layer BERT model, with an uncased vocab.
#           num_labels = 27, # The number of output labels--2 for binary classification.
#                     # You can increase this for multi-class tasks.   
#           output_attentions = False, # Whether the model returns attentions weights.
#           output_hidden_states = False,) # Whether the model returns all hidden-states.
        
        
# checkpoint = torch.load(camembert_model_path_desc)
# cam_desc.load_state_dict(checkpoint)

In [ ]:
# flau_title = vec_output_FlaubertForSequenceClassification.from_pretrained(
#         'flaubert/flaubert_base_cased', 
#         num_labels = 27, 
#         output_attentions = False,
#         output_hidden_states = False,)


# checkpoint = torch.load(flaubert_model_path_title)
# flau_title.load_state_dict(checkpoint)

In [ ]:
# flau_desc = vec_output_FlaubertForSequenceClassification.from_pretrained(
#         'flaubert/flaubert_base_cased', 
#         num_labels = 27, 
#         output_attentions = False,
#         output_hidden_states = False,)


# checkpoint = torch.load(flaubert_model_path_desc)
# flau_desc.load_state_dict(checkpoint)

#  Instantiation  & Training of Fusion Model 

In [ ]:
model = vector_fusion() 

In [ ]:
model.cuda()

# Fuse Input Data

In [ ]:
train_dataset = FusionDataset (train_df, tr_inputs_cam, tr_masks_cam, tr_inputs_flau, tr_masks_flau,
                            transform = image_transforms['train'])

val_dataset = FusionDataset (val_df, val_inputs_cam, val_masks_cam, val_inputs_flau, val_masks_flau,
                          transform = image_transforms['valid'])



test_dataset = FusionDataset (test_df, input_ids_test_cam, attention_masks_test_cam, input_ids_test_flau, attention_masks_test_flau
                           , transform = image_transforms['test'])

# Data Loaders

We need to use the DataLoaders to create iterable objects for us to work with. We tell it which datasets we want to use, give it a batch size, and shuffle the data

In [ ]:
batch_size = 32  #increase batch size to reduce the noise 

train_dataloader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)

validation_dataloader = DataLoader(val_dataset,batch_size=batch_size,shuffle=False)
 
test_dataloader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

In [ ]:
#Wj= n_samples/n_classes*n_samplesj

w1 = 10190/27* 374

w2 = 10190/27*571

w3 = 10190/27*202

w4 = 10190/27*584

w5 = 10190/27*331

w6 = 10190/27*599

w7 = 10190/27*311

w8 = 10190/27*609

w9 = 10190/27*248

w10 = 10190/27*516

w11 = 10190/27*573

w12 = 10190/27*321

w13 = 10190/27*1225

w14 = 10190/27*92

w15 = 10190/27*605

w16 = 10190/27*170

w17 = 10190/27*474

w18 = 10190/27*599

w19 = 10190/27*301

w20 = 10190/27*100

w21 = 10190/27*389

w22 = 10190/27*299

w23 = 10190/27*99

w24 = 10190/27*105

w25 = 10190/27*300

w26 = 10190/27*96

w27 = 10190/27*97

class_weights = torch.FloatTensor([w1, w2, w3, w4, w5, w6, w7, w8, w9, w10,
                                 w11, w12, w13, w14, w15, w16, w17, w18,
                                 w19, w20, w21, w22, w23, w24, w25, w26, w27]).cuda()

In [ ]:
import torch.nn as nn
loss_criterion = nn.CrossEntropyLoss(weight=class_weights)

#loss_criterion = CrossEntropyLossFlat(weight=class_weights)

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# test_loader = validation_dataloader

In [ ]:
ls -la

In [ ]:
# cp './Hirarical_add_concat_best_model.pt' '../drive/My Drive/Rakuten/models/'

# Model Testing

In [ ]:
model_path =  '/content/drive/My Drive/Rakuten/models/Final_Hirarical_model.pt'

In [ ]:
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint) # A state_dict is simply a Python dictionary object 
                                  # that maps each layer to its parameter tensor

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score


def predict_pyt(model, prediction_dataloader):
    """
    model: pytorch model
    prediction_dataloader: DataLoader object for which the predictions has to be made.
    return:
        predictions:    - Direct predicted labels
        softmax_logits: - logits which are normalized with softmax on output"""

    pred = []
    ground_truth = []
    
    print("")
    print("Running Testing...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    predictions=[]
    true_labels=[]
    logits_values =[]
    val_accuracy_values = []
    val_loss_values = []
    
    
    total_t0 = time.time()
    # Evaluate data for one epoch
    for batch in (prediction_dataloader):
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        
        b_img = batch[0].to(device)

        b_input_id_cam = batch[1].to(device)
        b_input_mask_cam = batch[2].to(device)
        b_input_id_flau = batch[3].to(device)
        b_input_mask_flau = batch[4].to(device)

        b_labels = batch[5].to(device)
        
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():       
        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.


            logits = model(b_img,b_input_id_cam ,b_input_mask_cam,b_input_id_flau,b_input_mask_flau)
            
        #new
        
        #defining the val loss
        loss = loss_criterion(logits, b_labels)
        
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()

        # Move logits and labels to CPU
        predicted_labels=np.argmax(logits,axis=1)
        predictions.extend(predicted_labels)
        label_ids = b_labels.to('cpu').numpy()

        true_labels.extend(label_ids)

        ##########################################################################

        logits_values.append(predicted_labels)

        ##########################################################################

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(prediction_dataloader)
    val_accuracy_values.append(avg_val_accuracy)
#--------------------------------
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(prediction_dataloader)
#-----------------------------
    val_loss_values.append(avg_val_loss)

    
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Test Loss: {0:.2f}".format(avg_val_loss))
    print("  Test took: {:}".format(validation_time))
    print("Test F1-Score: {}".format(f1_score(true_labels,predictions,average='macro')))
    curr_f1 = f1_score(true_labels,predictions,average='macro')

    # pred.append(predictions)
    # ground_truth.append(true_labels) 

    print("")
    print("Test complete!")

    # Results = pd.DataFrame(
    # {'Pred':    predictions,
    #  'Ground': true_labels
    # })

    # #print("Total testing took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
    # print()
    # plt.plot(np.array(val_accuracy_values), 'r', label='Test accuracy')
    # plt.legend()
    # plt.title('Test Curve')
    # plt.show()

    
    # print(Results)

    # Results.to_csv( "Baseline_80_20")

    print()
    print('DONE')

    return predictions, true_labels #, softmax_logits


In [ ]:
predictions_val, true_label = predict_pyt(model, test_dataloader)

In [ ]:
 Results = pd.DataFrame(
    {'Pred':    predictions_val,
     'Ground': true_label
    })



In [ ]:
Results.to_csv( "Hirachical_80_20")

In [ ]:
Results = pd.read_csv('Hirachical_80_20')

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Results['Ground'], Results['Pred'])

In [ ]:
print(f1_score(Results['Ground'],Results['Pred'],average='macro'))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_mat = confusion_matrix(Results['Ground'], Results['Pred'])

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

df_cm = pd.DataFrame(confusion_mat, index = [i for i in range(27)],
                                    columns = [i for i in range(27)])
plt.figure(figsize = (30,30))
sn.heatmap(df_cm, annot=True)

In [ ]:
from sklearn.metrics import classification_report

target_names = ['class 1', 'class 2', 'class 3', 'class 4', 'class 5', 'class 6',
                'class 7', 'class 8', 'class 9', 'class 10', 'class 11', 'class 12', 
                'class 13', 'class 14', 'class 15', 'class 16', 'class 17', 'class 18',
                'class 19','class 20', 'class 21', 'class 22', 'class 23', 'class 24', 'class 25', 'class 26', 'class 27']

print(classification_report(Results['Ground'], Results['Pred'], target_names=target_names))


In [ ]:
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier


n_classes = 27

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
# for i in range(n_classes):  
    # fpr[i], tpr[i], _ = roc_curve(true_label[:,i], predictions_val[:, i])
    # roc_auc[i] = auc(fpr[i], tpr[i])

fpr, tpr, _ = roc_curve(true_label, predictions_val)
roc_auc = auc(fpr, tpr)

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Plot of a ROC curve for a specific class
plt.figure()
plt.plot(fpr[2], tpr[2], label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

# Plot ROC curve
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]))

plt.plot(fpr, tpr, label='ROC curve of class {0} (area = {1:0.2f})'
                                   ''.format(i, roc_auc))

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

In [ ]:
print(true_label.type)
print(predictions_val.type)